In [1]:
# !pip install pyomo
# # Install GLPK solver package (includes glpsol executable)
# !apt-get install -y -qq glpk-utils
!pip install -q pyomo
!apt-get install -y -qq glpk-utils
from pyomo.environ import *
import itertools, random, math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# !apt-get install -y -qq coinor-cbc
# opt= SolverFactory('cbc', executable='/usr/bin/cbc')
# import os
# os.environ['NEOS_EMAIL'] = 'XXXXXX@gmail.com'
# from pyomo import environ as pe
# opt = pe.SolverManagerFactory('neos')

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 126675 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpacking glpk-utils (5.0-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4b

In [2]:
# ---------------------------
# Dados de exemplo (mude conforme necessário)
# ---------------------------
V = 3                   # número de veículos
T = 100                 # horizonte de tempo (t = 0..T)
L = 100                 # route length
lanes = 2               # pistas/lane
delta_s = 1             # variacao de velocidade
max_s_l = 10            # maximum speed for load vehicles
max_s_ul = 20           # maximum speed for unload vehicles
Rg = 10                  # granularity for vehicles’ position measurement
P = [1,2,3]             # plataformas
U = 3                   # number of platforms
K = 2                 # Loading/Unloading time = K * delta_t
rul = {1: [1, 1], 2: [5, 1], 3: [7, 1]} # posições das plataformas (discretas) todas plataformas estão na lane 1
dsafe = 1.0            # distância segura (m)
delta_t = 0.1           # duração do ciclo (s)

# Velocidades e acelerações discretas
A_vals = [-2, -1, 0, 1, 2]
S_vals = [i for i in range(-int(max_s_ul), int(max_s_ul)+1, int(delta_s))]


In [3]:
# ---------------------------
# 🔹 Geração estocástica de chegada de carga (Poisson)
# ---------------------------
# random.seed(42)
# np.random.seed(42)

# taxa média de chegada (λ por ciclo) para cada plataforma
lambda_rate = {1: 1., 2: 0.5, 3: 0.3}

c_u_t = {}
for u in P:
    for t in range(T):
        arrivals = np.random.poisson(lambda_rate[u])
        c_u_t[(u,t)] = 1 if arrivals >= 1 else 0

#c_u_t2 = {(u, t): int(np.random.poisson(lambda_rate[u]) >= 1) for u in ru for t in range(T)}

# imprimir o mapa de chegadas
print("Mapa de chegadas de carga (1=carga disponível):")
for u in P:
    row = [c_u_t[(u,t)] for t in range(10)]
    print(f"  Plataforma {u}: {row}")

# Condições iniciais (posições dos veículos)
# Geração uniforme entre 0 e track_length
p0 = {v: random.uniform(0, L) for v in range(1, V + 1)}
l0 = {v: random.randint(1,lanes) for v in range(1, V + 1)}
rvl = {v: [int(np.ceil(p0[v]/Rg)), l0[v]] for v in range(1, V + 1)}

# imprimir
print("Vehicls possition:")
for v in range(1,V+1):
    print(f"  Vehicle {v}: {p0[(v)]}, {l0[(v)]}, {rvl[(v)]}")

vLoad = {v: 1 if random.uniform(0,1)>0.5 else 0 for v in range(1,V+1)}

# imprimir
print("Vehicls cargo:")
for v in range(1,V+1):
    print(f"  Vehicle load {v}: {vLoad[(v)]}")

Mapa de chegadas de carga (1=carga disponível):
  Plataforma 1: [1, 1, 1, 1, 0, 1, 0, 0, 0, 0]
  Plataforma 2: [0, 0, 1, 1, 0, 0, 1, 0, 0, 0]
  Plataforma 3: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
Vehicls possition:
  Vehicle 1: 88.86933349190255, 1, [9, 1]
  Vehicle 2: 31.74311200472757, 2, [4, 2]
  Vehicle 3: 92.04390957893939, 1, [10, 1]
Vehicls cargo:
  Vehicle load 1: 0
  Vehicle load 2: 1
  Vehicle load 3: 0


In [ ]:
# ---------------------------
# Modelo Pyomo
# ---------------------------
model = ConcreteModel()

# Sets
model.V = RangeSet(1, V)
model.Lanes = RangeSet(1, lanes)
model.T = RangeSet(0, T-1)
model.U = RangeSet(1, U)
model.R = RangeSet(1, Rg)
model.K = RangeSet(0, K)

# Construir dicionário completo para Param com valores 0/1
rul_complete = {}
for u in model.U:
    for r in model.R:
        for lane in model.Lanes:
            if u in rul and rul[u] == [r, lane]:
                rul_complete[(u, r, lane)] = 1
            else:
                rul_complete[(u, r, lane)] = 0

# Construir dicionário completo para Param com valores 0/1
rvl_complete = {}
for v in model.V:
    for r in model.R:
        for lane in model.Lanes:
            if v in rvl and rvl[v] == [r, lane]:
                rvl_complete[(v, r, lane)] = 1
            else:
                rvl_complete[(v, r, lane)] = 0

# Parâmetros
model.rul = Param(model.U, model.R, model.Lanes, initialize=rul_complete)
model.rvl = Param(model.V, model.R, model.Lanes, initialize=rvl_complete)
model.cu = Param(model.U, model.T, initialize=c_u_t)
# model.dsafe = Param(initialize=dsafe)
model.delta_t = Param(initialize=delta_t)
model.delta_s = Param(initialize=delta_s)
model.sMaxL = Param(initialize=max_s_l)
model.sMax = Param(initialize=max_s_ul)
# model.L = Param(initialize=L)
# model.R_vals = Param(model.R, within=NonNegativeIntegers, initialize={ ... })

# Variáveis de decisão
model.p = Var(model.V, model.T, domain=NonNegativeReals, bounds=(0, L))   # posição
model.s = Var(model.V, model.T, domain=Reals)                              # velocidade
model.a = Var(model.V, model.T, domain=Integers, bounds=(-2, 2))           # aceleração discreta
model.r = Var(model.V, model.R, model.Lanes, model.T, domain=Binary)
model.c = Var(model.V, model.T, domain=Binary)                             # possui carga?
model.d = Var(model.V, model.T, domain=Binary)                             # entrega feita?


model.posBound = Var(model.V, model.T, domain=Binary)
model.at_platform = Var(model.V, model.T, model.U, domain=Binary)

# Initialize variables at t=0
for v in model.V:
  for k in model.K:
    model.p[v, k].fix(p0[v])
    # Assuming initial velocity is 0, you can change this if needed
    model.s[v, k].fix(0)
    model.c[v, k].fix(vLoad[v])
    model.d[v, k].fix(0)
    model.posBound[v, k]

# Initialize r at t=0 based on rvl
for v in model.V:
    for r in model.R:
        for lane in model.Lanes:
             model.r[v, r, lane, 0].fix(rvl_complete[(v, r, lane)])

In [ ]:
# -----------------------------
# Restrições
# -----------------------------

def one_hot_R_rule(m, v, lane, t):
    return sum(m.r[v, R, lane, t] for R in m.R) == 1
model.one_hot_r = Constraint(model.V, model.Lanes, model.T, rule=one_hot_R_rule)

def one_hot_l_rule(m, v, R, t):
    return sum(m.r[v, R, lane, t] for lane in m.Lanes) == 1
model.one_hot_l = Constraint(model.V, model.R, model.T, rule=one_hot_l_rule)

#Vinculação lógica para ativar o único R desejado
# def r_ceil_rule(m, v, R, lane, t):
#     # O valor alvo de ceil
#     r_ceil = (m.p[v, t] + R - 1) // R  # usando divisão inteira para simular ceil
#     # Só pode ser 1 se R for o correto
#     return m.r[v, R, lane, t] <= (r_ceil == R)
# model.r_ceil = Constraint(model.V, model.R, model.Lanes, model.T, rule=r_ceil_rule)

def r_lower_bound_rule(m, v, R, l, t):
    # R_aux = sum(1 - m.r[v, ra, l, t] for ra in m.R if ra <= R) + 1
    return m.r[v, R, l, t] * m.R[R] * L / Rg >= m.p[v, t]

def r_upper_bound_rule(m, v, R, l, t):
    epsilon = 1e-6
    return m.r[v, R, l, t] * (m.R[R]-1) + epsilon <= m.p[v, t]

model.r_lb = Constraint(model.V, model.R, model.Lanes, model.T, rule=r_lower_bound_rule)
model.r_ub = Constraint(model.V, model.R, model.Lanes, model.T, rule=r_upper_bound_rule)

# Atualização da velocidade: s_{v,t} = s_{v,t-1} + a_{v,t} * Δs
def speed_rule(m, v, t):
    if t == 0:
        return Constraint.Skip
    return m.s[v, t] == m.s[v, t-1] + m.a[v, t] * m.delta_s
model.speed_update = Constraint(model.V, model.T, rule=speed_rule)

# Atualização da posição: p_{v,t} = p_{v,t-1} + s_{v,t} * Δt (mod L)
def position_rule(m, v, t):
    if t == 0:
        return Constraint.Skip
    return m.p[v, t] == m.p[v, t-1] + m.s[v, t] * m.delta_t - L * m.posBound[v, t] #(m.p[v, t-1] + m.s[v, t] * m.delta_t) % L
model.pos_update = Constraint(model.V, model.T, rule=position_rule)

def pos_bounds_rule(m, v, t):
    return (0, m.p[v, t], L)
model.p_bounds = Constraint(model.V, model.T, rule=pos_bounds_rule)

# Restrições de segurança: distância mínima entre veículos
# def safety_rule1(m, v1, v2, t):
#     if v1 >= v2:
#         return Constraint.Skip
#     return m.p[v1, t] - m.p[v2, t] >= m.dsafe
# model.safety = Constraint(model.V, model.V, model.T, rule=safety_rule1)

# def safety_rule2(m, v1, v2, t):
#     if v1 >= v2:
#         return Constraint.Skip
#     return m.p[v2, t] - m.p[v1, t] >= m.dsafe
# model.safety = Constraint(model.V, model.V, model.T, rule=safety_rule2)

# Limites de velocidade conforme carga
def speed_limit_rule(m, v, t):
    return m.s[v, t] <= m.sMaxL + (1 - m.c[v, t]) * (m.sMax - m.sMaxL)
model.speed_limit = Constraint(model.V, model.T, rule=speed_limit_rule)

# Carga/descarga simples (heurística)
# def delivery_possible_rule(m, v, t):
#     # simplificação: se o veículo está na posição de alguma plataforma e há carga
#     expr = sum((abs(m.p[v, t] - m.ru[u]) <= 1) * m.cu[u, t] for u in m.U)
#     return m.d[v, t] <= expr
# model.delivery = Constraint(model.V, model.T, rule=delivery_possible_rule)

# Enforce at_platform[v, t, u] = 1 iff abs(m.p[v, t] - m.ru[u]) <= 1
def at_platform_rule(m, v, r, l, t, u):
    return m.r[v, r, l, t] + m.rul[u, r , l] <= 1 + m.at_platform[v, t, u]
model.platform_ub = Constraint(model.V, model.R, model.Lanes, model.T, model.U, rule=at_platform_rule)

# Introduce auxiliary binary variable z[v, t, u] = at_platform[v, t, u] * c[v, t]
model.z = Var(model.V, model.T, model.U, domain=Binary)

# Linearized constraints for z[v, t, u] = at_platform[v, t, u] * c[v, t]
def z_linearization_rule1(m, v, t, u):
    return m.z[v, t, u] <= m.at_platform[v, t, u]
model.z_linearization1 = Constraint(model.V, model.T, model.U, rule=z_linearization_rule1)

def z_linearization_rule2(m, v, t, u):
    return m.z[v, t, u] <= m.c[v, t]
model.z_linearization2 = Constraint(model.V, model.T, model.U, rule=z_linearization_rule2)

def z_linearization_rule3(m, v, t, u):
    return m.z[v, t, u] >= m.at_platform[v, t, u] + m.c[v, t] - 1
model.z_linearization3 = Constraint(model.V, model.T, model.U, rule=z_linearization_rule3)


def delivery_possible_rule(m, v, t):
    if t-K < 0:
        return Constraint.Skip
    # Use the auxiliary variable z instead of the product
    expr = sum(model.z[v, t-k, u] for k in model.K for u in model.U)
    return model.d[v, t] * (K + 1) <= expr # Multiply by K+1 to match the original sum logic
model.delivery = Constraint(model.V, model.T, rule=delivery_possible_rule)

In [ ]:
# -----------------------------
# Função objetivo
# -----------------------------
def objective_rule(m):
    return sum(m.d[v, t] for v in m.V for t in m.T)
model.obj = Objective(rule=objective_rule, sense=maximize)

In [ ]:
# -----------------------------
# Solver
# -----------------------------
# solver = SolverFactory("glpk")
# results = solver.solve(model, tee=True)

solver = SolverFactory('glpk', executable='/usr/bin/glpsol')
results = solver.solve(model, tee=True)

# -----------------------------
# Resultados
# -----------------------------
print("\n=== RESULTADOS ===")
for v in model.V:
    print(f"Veículo {v}:")
    for t in model.T:
        print(f" t={t:2d} | pos={value(model.p[v,t]):6.2f} | vel={value(model.s[v,t]):6.2f} | carga={int(value(model.c[v,t]))} | entrega={int(value(model.d[v,t]))}")

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmpq93i5fl6.glpk.raw --wglp /tmp/tmp27ry0xbr.glpk.glp --cpxlp
 /tmp/tmpb43ygtyj.pyomo.lp
Reading problem data from '/tmp/tmpb43ygtyj.pyomo.lp'...
/tmp/tmpb43ygtyj.pyomo.lp:207573: warning: lower bound of variable 'x2' redefined
/tmp/tmpb43ygtyj.pyomo.lp:207573: warning: upper bound of variable 'x2' redefined
38088 rows, 9499 columns, 83016 non-zeros
8916 integer variables, 8619 of which are binary
216192 lines were read
Writing problem data to '/tmp/tmp27ry0xbr.glpk.glp'...
169268 lines were written
GLPK Integer Optimizer 5.0
38088 rows, 9499 columns, 83016 non-zeros
8916 integer variables, 8619 of which are binary
Preprocessing...
PROBLEM HAS NO PRIMAL FEASIBLE SOLUTION
Time used:   0.0 secs
Memory used: 19.3 Mb (20284220 bytes)
Writing MIP solution to '/tmp/tmpq93i5fl6.glpk.raw'...
47596 lines were written


ERROR:pyomo.common.numeric_types:evaluating object as numeric value: p[1,3]
    (object: <class 'pyomo.core.base.var.VarData'>)
No value for uninitialized VarData object p[1,3]



=== RESULTADOS ===
Veículo 1:
 t= 0 | pos= 40.67 | vel=  0.00 | carga=0 | entrega=0
 t= 1 | pos= 40.67 | vel=  0.00 | carga=0 | entrega=0
 t= 2 | pos= 40.67 | vel=  0.00 | carga=0 | entrega=0


ValueError: No value for uninitialized VarData object p[1,3]

In [ ]:
results = solver.solve(model, tee=True)
results.write()  # para ver status
print(results.solver.status)
print(results.solver.termination_condition)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmpgmj345w4.glpk.raw --wglp /tmp/tmpqji0iorj.glpk.glp --cpxlp
 /tmp/tmpqe7t8cgg.pyomo.lp
Reading problem data from '/tmp/tmpqe7t8cgg.pyomo.lp'...
/tmp/tmpqe7t8cgg.pyomo.lp:207573: warning: lower bound of variable 'x2' redefined
/tmp/tmpqe7t8cgg.pyomo.lp:207573: warning: upper bound of variable 'x2' redefined
38088 rows, 9499 columns, 83016 non-zeros
8916 integer variables, 8619 of which are binary
216192 lines were read
Writing problem data to '/tmp/tmpqji0iorj.glpk.glp'...
169268 lines were written
GLPK Integer Optimizer 5.0
38088 rows, 9499 columns, 83016 non-zeros
8916 integer variables, 8619 of which are binary
Preprocessing...
PROBLEM HAS NO PRIMAL FEASIBLE SOLUTION
Time used:   0.0 secs
Memory used: 19.3 Mb (20284220 bytes)
Writing MIP solution to '/tmp/tmpgmj345w4.glpk.raw'...
47596 lines were written
# ==========================================================
# = Solver Results            